# WRC Live Timing API Scraper

Simple exploration of grabbing data from WRC Live Timing API.

TO DO - how do we lookup `championshipId`?

In [418]:
from urllib.parse import urljoin
from parse import parse
import pathlib

import pandas as pd
import requests

#%pip install requests-cache
import requests_cache
requests_cache.install_cache("demo_cache2")

In [535]:
CURRENT_YEAR = 2024

WRC_API_BASE = "https://api.wrc.com/content/{path}"

In [536]:
from itertools import zip_longest


def _WRC_json(path, base=WRC_API_BASE, retUrl=False):
    """Return JSON from API."""
    url = urljoin(base, path)
    if retUrl:
        return url
    # print(f"Fetching: {url}")
    r = requests.get(url)
    rj = r.json()
    if "status" in rj and rj["status"]=="Not Found":
        return {}
    return r.json()


def convert_date_range(date_range_str):
    """Convert date of from `19 - 22 JAN 2023` to date range."""
    r = parse("{start_day} - {end_day} {month} {year}", date_range_str)
    start_date = pd.to_datetime(
        f"{r['start_day']} {r['month']} {r['year']}", format="%d %b %Y"
    )
    end_date = pd.to_datetime(
        f"{r['end_day']} {r['month']} {r['year']}", format="%d %b %Y"
    )
    return pd.date_range(start=start_date, end=end_date)


def timeify(df, col, typ=None):
    """Convert a column  to a datetime inplace."""
    if typ == "daterange":
        df[col] = df[col].apply(convert_date_range)
    else:
        df[col] = pd.to_datetime(df[col].astype(int), unit="ms")


def tablify(json_data, subcolkey=None):
    """Generate table from separate colnames/values JSON."""
    # Note that the JSON may be a few rows short cf. provided keys
    if subcolkey is None:
        results = []
        for values in json_data["values"]:
            result_dict = {}
            # Zip keys and values, filling None for missing values
            zipped_values = zip_longest(json_data["fields"], values, fillvalue=None)
            # Convert the zipped values to a dictionary and update the result_dict
            result_dict.update(dict(zipped_values))
            results.append(result_dict)
        return pd.DataFrame(results)
    else:
        df = pd.DataFrame(columns=json_data["fields"])
        for value in json_data["values"]:
            _df = pd.DataFrame(value[subcolkey])
            if len(_df.columns) < len(json_data["fields"]):
                _df[[json_data["fields"][len(_df.columns) :]]] = None
            _df.columns = json_data["fields"]
            for c in [k for k in value.keys() if k != subcolkey]:
                _df[c] = value[c]
                df = pd.concat([df, _df])
        return df

def stage_id_annotations(df, eventId=None, rallyId=None, stageId=None):
    if "eventId" not in df.columns:
        df["eventId"] = eventId
    if "rallyId" not in df.columns:
        df["rallyId"] = rallyId
    if "stageId" not in df.columns:
        df["stageId"] = stageId

## Season Calendar

The API call `https://api.wrc.com/content/filters/calendar` gives a list of current or latest (last, if season's end) events, from which we can get a season identifer. For the 2023 WRC championship, the season ID is `20`

In [537]:
"""
https://www.wrc.com/live-timing?liveTimingMenu=overall_livetiming&stage=FINAL&championshipId=249
<div id="dropdown" style="max-height: 92.25px;">
<div data-key="245" class="sc-dVhcbM hzWUOI">ALL</div>
<div data-key="247" class="sc-dVhcbM hzWUOI">WRC</div>
<div data-key="249" class="sc-dVhcbM hzWUOI" active="true">WRC2</div>
<div data-key="256" class="sc-dVhcbM hzWUOI">WRC3</div>
<div data-key="258" class="sc-dVhcbM hzWUOI">Junior WRC</div>
<div data-key="251" class="sc-dVhcbM hzWUOI">WRC2 Challenger</div>
<div data-key="254" class="sc-dVhcbM hzWUOI">Master Cup</div></div>

"""

championshipIds = {"all": 245,
                   "wrc": 247,
                   "wrc2": 249,
                   "wrc3": 256,
                   "jwrc": 258,
                   "wrc2c": 251,
                   "mcup": 254}

championshipId = championshipIds["wrc"]

In [538]:
stub = f"filters/calendar?language=en&size=20&championship=wrc&origin=vcms&year=2024"
json_data = _WRC_json(stub)
keys_to_keep = ['id', 'guid', 'title', 'location', 'startDate', 'endDate', 'eventId',
       'rallyId', 'description', 'round', 'cvpSeriesLink', 'sponsor', 'images',
       'season', 'competition', 'country', 'asset', '__typename', 'type',
       'uid', 'seriesUid', 'releaseYear', 'availableOn', 'availableTill',
       'startDateLocal', 'endDateLocal', 'finishDate', 'championship',
       'championshipLogo'
]
filtered_data = [
    {key: item[key] for key in keys_to_keep if key in item}
    for item in json_data["content"]
]

pd.DataFrame(filtered_data)

,id,guid,title,location,startDate,endDate,eventId,rallyId,description,round,...,uid,seriesUid,releaseYear,availableOn,availableTill,startDateLocal,endDateLocal,finishDate,championship,championshipLogo
0,C87D,WRC_2024_01,Rallye Monte-Carlo,Monaco,1706110200000,1706454000000,446,477,The most unpredictable rally of the year. Rela...,1,...,C87D,WRC_2024_01,2024,1706110200000,1706454000000,2024-01-24T16:30:00+01:00,2024-01-28T16:00:00+01:00,1706454000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
1,ANhw,WRC_2024_02,WRC Rally Sweden,sweden,1707984000000,1708268400000,447,478,The WRC’s ultimate winter challenge. Watch in ...,2,...,ANhw,WRC_2024_02,2024,1707984000000,1708268400000,2024-02-15T09:00:00+01:00,2024-02-18T16:00:00+01:00,1708268400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
2,tv2t,WRC_2024_03,WRC Safari Rally Kenya,"Naivasha, Kenya",1711522800000,1711890000000,448,479,Held on the untamed terrains of Africa’s breat...,3,...,tv2t,WRC_2024_03,2024,1711522800000,1711890000000,2024-03-27T10:00:00+03:00,2024-03-31T16:00:00+03:00,1711890000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
3,JCpO,WRC_2024_04,WRC Croatia Rally,"Zagreb, City of Zagreb Region",1713423600000,1713708000000,449,480,High-speed stretches meet challenging hairpin ...,4,...,JCpO,WRC_2024_04,2024,1713423600000,1713708000000,2024-04-18T09:00:00+02:00,2024-04-21T16:00:00+02:00,1713708000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
4,lLrv,WRC_2024_05,WRC Vodafone Rally de Portugal,"Matosinhos, Porto",1715238000000,1715522400000,450,481,Fast but technical gravel roads inland from Po...,5,...,lLrv,WRC_2024_05,2024,1715238000000,1715522400000,2024-05-09T08:00:00+01:00,2024-05-12T15:00:00+01:00,1715522400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
5,5VRr,WRC_2024_06,WRC Rally Italia Sardegna,"Olbia, Sardinia",1717087500000,1717329600000,452,483,Don’t let the picturesque Mediterranean backdr...,6,...,5VRr,WRC_2024_06,2024,1717087500000,1717329600000,2024-05-30T18:45:00+02:00,2024-06-02T14:00:00+02:00,1717329600000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
6,htOR,WRC_2024_07,WRC ORLEN 80th Rally Poland,Poland,1719475200000,1719756000000,453,484,Back on the WRC calendar! Poland's lightning-f...,7,...,htOR,WRC_2024_07,2024,1719475200000,1719756000000,2024-06-27T10:00:00+02:00,2024-06-30T16:00:00+02:00,1719756000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
7,228Z,WRC_2024_08,WRC Tet Rally Latvia,"Liepāja, Latvia",1721277060000,1721566800000,454,485,WRC is thrilled to welcome a newcomer in Tet R...,8,...,228Z,WRC_2024_08,2024,1721277060000,1721566800000,2024-07-18T07:31:00+03:00,2024-07-21T16:00:00+03:00,1721566800000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
8,Dv15,WRC_2024_09,WRC Secto Rally Finland,Jyväskylä,1722441600000,1722772800000,455,486,A mecca for rally drivers and enthusiasts alik...,9,...,Dv15,WRC_2024_09,2024,1722441600000,1722772800000,2024-07-31T19:00:00+03:00,2024-08-04T15:00:00+03:00,1722772800000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
9,csx1,WRC_2024_10,WRC EKO Acropolis Rally Greece,Lamia,1725516060000,1725796800000,456,487,A legendary WRC fixture. Twisty gravel mountai...,10,...,csx1,WRC_2024_10,2024,1725516060000,1725796800000,2024-09-05T09:01:00+03:00,2024-09-08T15:00:00+03:00,1725796800000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...


In [539]:
def getFullCalendar(year=CURRENT_YEAR, championship="wrc", size=20):
    stub = f"filters/calendar?language=en&size={size}&championship={championship}&origin=vcms&year={year}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.dataFrame()
    # return json_data
    return pd.DataFrame(json_data["content"])


df_fullcal = getFullCalendar()
df_fullcal.head()

,id,guid,title,location,startDate,endDate,eventId,rallyId,description,round,...,uid,seriesUid,releaseYear,availableOn,availableTill,startDateLocal,endDateLocal,finishDate,championship,championshipLogo
0,C87D,WRC_2024_01,Rallye Monte-Carlo,Monaco,1706110200000,1706454000000,446,477,The most unpredictable rally of the year. Rela...,1,...,C87D,WRC_2024_01,2024,1706110200000,1706454000000,2024-01-24T16:30:00+01:00,2024-01-28T16:00:00+01:00,1706454000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
1,ANhw,WRC_2024_02,WRC Rally Sweden,sweden,1707984000000,1708268400000,447,478,The WRC’s ultimate winter challenge. Watch in ...,2,...,ANhw,WRC_2024_02,2024,1707984000000,1708268400000,2024-02-15T09:00:00+01:00,2024-02-18T16:00:00+01:00,1708268400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
2,tv2t,WRC_2024_03,WRC Safari Rally Kenya,"Naivasha, Kenya",1711522800000,1711890000000,448,479,Held on the untamed terrains of Africa’s breat...,3,...,tv2t,WRC_2024_03,2024,1711522800000,1711890000000,2024-03-27T10:00:00+03:00,2024-03-31T16:00:00+03:00,1711890000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
3,JCpO,WRC_2024_04,WRC Croatia Rally,"Zagreb, City of Zagreb Region",1713423600000,1713708000000,449,480,High-speed stretches meet challenging hairpin ...,4,...,JCpO,WRC_2024_04,2024,1713423600000,1713708000000,2024-04-18T09:00:00+02:00,2024-04-21T16:00:00+02:00,1713708000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
4,lLrv,WRC_2024_05,WRC Vodafone Rally de Portugal,"Matosinhos, Porto",1715238000000,1715522400000,450,481,Fast but technical gravel roads inland from Po...,5,...,lLrv,WRC_2024_05,2024,1715238000000,1715522400000,2024-05-09T08:00:00+01:00,2024-05-12T15:00:00+01:00,1715522400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...


In [540]:
len(df_fullcal)

13

In [541]:
df_fullcal.columns

Index(['id', 'guid', 'title', 'location', 'startDate', 'endDate', 'eventId',
       'rallyId', 'description', 'round', 'cvpSeriesLink', 'sponsor', 'images',
       'season', 'competition', 'country', 'asset', '__typename', 'type',
       'uid', 'seriesUid', 'releaseYear', 'availableOn', 'availableTill',
       'startDateLocal', 'endDateLocal', 'finishDate', 'championship',
       'championshipLogo'],
      dtype='object')

In [542]:
df_fullcal

,id,guid,title,location,startDate,endDate,eventId,rallyId,description,round,...,uid,seriesUid,releaseYear,availableOn,availableTill,startDateLocal,endDateLocal,finishDate,championship,championshipLogo
0,C87D,WRC_2024_01,Rallye Monte-Carlo,Monaco,1706110200000,1706454000000,446,477,The most unpredictable rally of the year. Rela...,1,...,C87D,WRC_2024_01,2024,1706110200000,1706454000000,2024-01-24T16:30:00+01:00,2024-01-28T16:00:00+01:00,1706454000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
1,ANhw,WRC_2024_02,WRC Rally Sweden,sweden,1707984000000,1708268400000,447,478,The WRC’s ultimate winter challenge. Watch in ...,2,...,ANhw,WRC_2024_02,2024,1707984000000,1708268400000,2024-02-15T09:00:00+01:00,2024-02-18T16:00:00+01:00,1708268400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
2,tv2t,WRC_2024_03,WRC Safari Rally Kenya,"Naivasha, Kenya",1711522800000,1711890000000,448,479,Held on the untamed terrains of Africa’s breat...,3,...,tv2t,WRC_2024_03,2024,1711522800000,1711890000000,2024-03-27T10:00:00+03:00,2024-03-31T16:00:00+03:00,1711890000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
3,JCpO,WRC_2024_04,WRC Croatia Rally,"Zagreb, City of Zagreb Region",1713423600000,1713708000000,449,480,High-speed stretches meet challenging hairpin ...,4,...,JCpO,WRC_2024_04,2024,1713423600000,1713708000000,2024-04-18T09:00:00+02:00,2024-04-21T16:00:00+02:00,1713708000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
4,lLrv,WRC_2024_05,WRC Vodafone Rally de Portugal,"Matosinhos, Porto",1715238000000,1715522400000,450,481,Fast but technical gravel roads inland from Po...,5,...,lLrv,WRC_2024_05,2024,1715238000000,1715522400000,2024-05-09T08:00:00+01:00,2024-05-12T15:00:00+01:00,1715522400000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
5,5VRr,WRC_2024_06,WRC Rally Italia Sardegna,"Olbia, Sardinia",1717087500000,1717329600000,452,483,Don’t let the picturesque Mediterranean backdr...,6,...,5VRr,WRC_2024_06,2024,1717087500000,1717329600000,2024-05-30T18:45:00+02:00,2024-06-02T14:00:00+02:00,1717329600000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
6,htOR,WRC_2024_07,WRC ORLEN 80th Rally Poland,Poland,1719475200000,1719756000000,453,484,Back on the WRC calendar! Poland's lightning-f...,7,...,htOR,WRC_2024_07,2024,1719475200000,1719756000000,2024-06-27T10:00:00+02:00,2024-06-30T16:00:00+02:00,1719756000000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
7,228Z,WRC_2024_08,WRC Tet Rally Latvia,"Liepāja, Latvia",1721277060000,1721566800000,454,485,WRC is thrilled to welcome a newcomer in Tet R...,8,...,228Z,WRC_2024_08,2024,1721277060000,1721566800000,2024-07-18T07:31:00+03:00,2024-07-21T16:00:00+03:00,1721566800000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
8,Dv15,WRC_2024_09,WRC Secto Rally Finland,Jyväskylä,1722441600000,1722772800000,455,486,A mecca for rally drivers and enthusiasts alik...,9,...,Dv15,WRC_2024_09,2024,1722441600000,1722772800000,2024-07-31T19:00:00+03:00,2024-08-04T15:00:00+03:00,1722772800000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...
9,csx1,WRC_2024_10,WRC EKO Acropolis Rally Greece,Lamia,1725516060000,1725796800000,456,487,A legendary WRC fixture. Twisty gravel mountai...,10,...,csx1,WRC_2024_10,2024,1725516060000,1725796800000,2024-09-05T09:01:00+03:00,2024-09-08T15:00:00+03:00,1725796800000,WRC,[{'url': 'https://wrc-static.enhance.diagnal.c...


Let's get the `seasonId` from a season record in the `season` column:

In [543]:
df_fullcal["season"][0]

{'id': '3b9e27c7-5cce-4264-8a63-1dc5f7b52017',
 'name': None,
 'seasonId': '28',
 'title': '2024',
 '__typename': 'Season'}

In [544]:
CURRENT_SEASON = df_fullcal["season"][0]["seasonId"]

seasonId = CURRENT_SEASON

# Check
# assert int(CURRENT_SEASON)==28 if CURRENT_YEAR==2024 else False
# And / or check the title?
# assert df_fullcal["season"][0]['title'] == '2024'

CURRENT_SEASON

'28'

In [545]:
# https://api.wrc.com/content/result/calendar?season=20&championship=wrc

def getResultsCalendar(seasonId=CURRENT_SEASON,  retUrl=False):
    """Get the WRC Calendar for a given season ID as a JSON result."""
    stub = f"result/calendar?season={seasonId}&championship=wrc"
    if retUrl:
        return stub
    json_data = _WRC_json(stub)
    df_calendar = tablify(json_data)
    #timeify(df_calendar, "date", "daterange")
    #timeify(df_calendar, "startDate")
    #timeify(df_calendar, "finishDate")
    # df_calendar.set_index("id", inplace=True)
    return df_calendar

In [546]:
getResultsCalendar().columns

Index(['id', 'rallyTitle', 'ROUND', 'rallyCountry', 'rallyCountryImage',
       'rallyId', 'date', 'startDate', 'finishDate', 'driverId',
       'driverCountryImage', 'driver', 'coDriverId', 'coDriverCountryImage',
       'coDriver', 'teamId', 'teamLogo', 'teamName', 'manufacturer'],
      dtype='object')

In [547]:
getResultsCalendar(retUrl=True)

'result/calendar?season=28&championship=wrc'

In [548]:
getResultsCalendar().head()

,id,rallyTitle,ROUND,rallyCountry,rallyCountryImage,rallyId,date,startDate,finishDate,driverId,driverCountryImage,driver,coDriverId,coDriverCountryImage,coDriver,teamId,teamLogo,teamName,manufacturer
0,C87D,Rallye Monte-Carlo,1,Monaco,Flags/MCO.png,477,24 - 28 JAN 2024,1706110200000,1706454000000,c99a2a26-bd03-5153-aaa7-684d3acb5491,Flags/BEL.png,Thierry NEUVILLE,b1b98699-0332-528a-8a80-11eb538f1ded,Flags/BEL.png,Martijn WYDAEGHE,b6692ea5-df92-5cad-a91c-20319a6fffd7,Flags/hyundai.png,HYUNDAI SHELL MOBIS WORLD RALLY TEAM,Hyundai
1,ANhw,WRC Rally Sweden,2,Sweden,Flags/SWE.png,478,15 - 18 FEB 2024,1707984000000,1708268400000,916f5b10-fee9-5b4c-b17b-6fbbc343cc3c,Flags/FIN.png,Esapekka LAPPI,2ef28d31-4a74-5794-b720-12c327c941b6,Flags/FIN.png,Janne FERM,b6692ea5-df92-5cad-a91c-20319a6fffd7,Flags/hyundai.png,HYUNDAI SHELL MOBIS WORLD RALLY TEAM,Hyundai
2,tv2t,WRC Safari Rally Kenya,3,Kenya,Flags/KEN.png,479,27 - 31 MAR 2024,1711522800000,1711890000000,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Flags/FIN.png,Kalle ROVANPERÄ,e4dd8a3f-00e9-59f7-9871-9337af6085d7,Flags/FIN.png,Jonne HALTTUNEN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota
3,JCpO,WRC Croatia Rally,4,Croatia,Flags/HRV.png,480,18 - 21 APR 2024,1713423600000,1713708000000,1cf9ade6-25b5-5586-8393-eacfeb943eae,Flags/FRA.png,Sébastien OGIER,867cd58f-3fe3-5290-9cc5-b21cb41c523c,Flags/FRA.png,Vincent LANDAIS,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota
4,lLrv,WRC Vodafone Rally de Portugal,5,Portugal,Flags/PRT.png,481,09 - 12 MAY 2024,1715238000000,1715522400000,1cf9ade6-25b5-5586-8393-eacfeb943eae,Flags/FRA.png,Sébastien OGIER,867cd58f-3fe3-5290-9cc5-b21cb41c523c,Flags/FRA.png,Vincent LANDAIS,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota


In [549]:
getResultsCalendar().tail()

,id,rallyTitle,ROUND,rallyCountry,rallyCountryImage,rallyId,date,startDate,finishDate,driverId,driverCountryImage,driver,coDriverId,coDriverCountryImage,coDriver,teamId,teamLogo,teamName,manufacturer
8,Dv15,WRC Secto Rally Finland,9,Finland,Flags/FIN.png,486,31 - 04 AUG 2024,1722441600000,1722772800000,1cf9ade6-25b5-5586-8393-eacfeb943eae,Flags/FRA.png,Sébastien OGIER,867cd58f-3fe3-5290-9cc5-b21cb41c523c,Flags/FRA.png,Vincent LANDAIS,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota
9,csx1,WRC EKO Acropolis Rally Greece,10,Greece,Flags/GRC.png,487,05 - 08 SEP 2024,1725516060000,1725796800000,c99a2a26-bd03-5153-aaa7-684d3acb5491,Flags/BEL.png,Thierry NEUVILLE,b1b98699-0332-528a-8a80-11eb538f1ded,Flags/BEL.png,Martijn WYDAEGHE,b6692ea5-df92-5cad-a91c-20319a6fffd7,Flags/hyundai.png,HYUNDAI SHELL MOBIS WORLD RALLY TEAM,Hyundai
10,D8sQ,WRC Rally Chile Bio Bío,11,Chile,Flags/CHL.png,488,26 - 29 SEP 2024,1727350260000,1727632800000,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Flags/FIN.png,Kalle ROVANPERÄ,e4dd8a3f-00e9-59f7-9871-9337af6085d7,Flags/FIN.png,Jonne HALTTUNEN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota
11,I8c3,WRC Central European Rally,12,Europe,Flags/EUR.png,489,17 - 20 OCT 2024,1729150200000,1729431000000,6632e7ca-34bf-55b8-9cad-d060000fa794,Flags/EST.png,Ott TÄNAK,00a8a5c3-f7ba-5086-86df-1a59b7da7e26,Flags/EST.png,Martin JÄRVEOJA,b6692ea5-df92-5cad-a91c-20319a6fffd7,Flags/hyundai.png,HYUNDAI SHELL MOBIS WORLD RALLY TEAM,Hyundai
12,G6SR,WRC FORUM8 Rally Japan,13,Japan,Flags/JPN.png,490,21 - 24 NOV 2024,1732147200000,1732431600000,ae7329c9-79b3-5d96-886c-22cca6217764,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,Flags/GBR.png,Scott MARTIN,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Flags/toyota.png,TOYOTA GAZOO RACING WRT,Toyota


In [550]:
import datetime

def timeNow(typ="ms"):
    now = int(datetime.datetime.now().timestamp())
    if typ=="ms":
        now *=1000
    return now

timeNow()

1732739621000

In [551]:
eventId, rallyId = df_fullcal[df_fullcal["startDate"] < timeNow()].iloc[-1][
    ["eventId", "rallyId"]
]
eventId, rallyId

('459', '490')

In [552]:
## Stages


def getStageDetails(eventId, rallyId):
    stub = f"result/stages?eventId={eventId}&rallyId={rallyId}&championship=wrc"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.dataFrame()
    df_stageDetails = tablify(json_data)
    return df_stageDetails


df_stageDetails = getStageDetails(eventId, rallyId)
df_stageDetails.head(3)

,id,STAGE,STAGE TYPE,stageId,eventId,STATUS,day,name,distance
0,ee55daa1-fd07-574f-835a-f39008081b95,SHD,shakedown,SHD,459,Complete,,Shakedown,
1,5f1d03f2-89e9-54c5-b8df-8643a383589d,SS1,HeadToHeadSuperSpecialStage,8175,459,Completed,Thursday,SS1 TOYOTA STADIUM SSS 1 (2.15km),2.15
2,1b14eff4-7157-5da3-925c-005d6b77cd55,SS2,SpecialStage,8176,459,Completed,Friday,SS2 Isegami's Tunnel 1 (23.67km),23.67


In [553]:
df_stageDetails.tail()

,id,STAGE,STAGE TYPE,stageId,eventId,STATUS,day,name,distance
18,b48f6f67-73aa-550c-9b9e-bba415e1922e,SS18,SpecialStage,8192,459,Completed,Sunday,SS18 Lake Mikawako 1 (13.98km),13.98
19,9626db05-e032-51eb-9d24-3d19fccdbd85,SS19,SpecialStage,8193,459,Completed,Sunday,SS19 Nukata 2 (20.23km),20.23
20,1c918158-c239-5da1-992d-9199dd15a825,SS20,HeadToHeadSuperSpecialStage,8194,459,Completed,Sunday,SS20 TOYOTA STADIUM SSS 3 (2.15km),2.15
21,d3063b6a-09e5-5a6f-b574-5bb4bdf557c8,SS21,PowerStage,8195,459,Completed,Sunday,SS21 Lake Mikawako 2 Wolf Power Stage (13.98km),13.98
22,f8f13d04-bbaf-58d1-906d-c2d459999c1b,FINAL,overall result,FINAL,459,Completed,,Final Result,


In [554]:
def getOverall(eventId, rallyId, stageId, championship="wrc", group="all"):
    stub = f"result/stageResult?eventId={eventId}&rallyId={rallyId}&stageId={stageId}&championship={championship}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_overall = tablify(json_data)
    stage_id_annotations(df_overall, eventId, rallyId, stageId)
    if group=="all":
        return df_overall
    else:
        return df_overall[df_overall["groupClass"]==group]

stageId = df_stageDetails[df_stageDetails["stageId"]!="FINAL"].iloc[-1]["stageId"]

getOverall(eventId, rallyId, stageId)#.head(3)

,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,penaltyTime,time,totalTime,diffFirst,diffPrev,groupClass,championshipId,eventId,rallyId,stageId
0,08000243-2cce-5380-aa40-77e11c0ee5f8,1,#21,ffedfbf6-eea2-5322-abaf-61c1950f955b,Bulgaria,Flags/BGR.png,Nikolay GRYAZIN,7d63ec40-0bb6-5da3-954e-846a2d40a9a1,ANA,Flags/ANA.png,...,,3:33:45.3,3:33:45.3,,,RC2,250,459,490,8195
1,32fe15b2-3330-5ce8-89fe-4c2838fc0e4d,2,#21,ffedfbf6-eea2-5322-abaf-61c1950f955b,Bulgaria,Flags/BGR.png,Nikolay GRYAZIN,7d63ec40-0bb6-5da3-954e-846a2d40a9a1,ANA,Flags/ANA.png,...,,3:33:45.3,3:33:45.3,,,RC2,249,459,490,8195
2,5bb89c25-e9fd-5c52-9328-780d9f822fa0,3,#21,ffedfbf6-eea2-5322-abaf-61c1950f955b,Bulgaria,Flags/BGR.png,Nikolay GRYAZIN,7d63ec40-0bb6-5da3-954e-846a2d40a9a1,ANA,Flags/ANA.png,...,,3:33:45.3,3:33:45.3,,,RC2,251,459,490,8195
3,a62da71c-e580-54b5-87cc-b8d095621e06,4,#21,ffedfbf6-eea2-5322-abaf-61c1950f955b,Bulgaria,Flags/BGR.png,Nikolay GRYAZIN,7d63ec40-0bb6-5da3-954e-846a2d40a9a1,ANA,Flags/ANA.png,...,,3:33:45.3,3:33:45.3,,,RC2,252,459,490,8195
4,e2051a07-247e-5414-a393-4f277999da5d,5,#33,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,...,,3:23:41.0,3:23:41.0,+-10:-4.-3,+-10:-4.-3,RC1,246,459,490,8195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,8ade2e63-9e53-5faa-8f08-d442a04fb61a,96,#34,41e224a3-19dd-5807-add2-33de34f93081,Japan,Flags/JPN.png,Norihiko KATSUTA,cb36335d-dc40-52bd-9ad4-70679ee337f5,Japan,Flags/JPN.png,...,,3:45:08.8,3:45:08.8,+11:23.5,+-32:-28.-2,RC2,245,459,490,8195
96,1cd7163c-c401-5846-96b9-8a3b24d0b6f0,97,#34,41e224a3-19dd-5807-add2-33de34f93081,Japan,Flags/JPN.png,Norihiko KATSUTA,cb36335d-dc40-52bd-9ad4-70679ee337f5,Japan,Flags/JPN.png,...,,3:45:08.8,3:45:08.8,+11:23.5,,RC2,246,459,490,8195
97,bd92f326-7297-547b-ba61-4164100209b9,98,#28,15aa0c7e-8fbd-5923-9b70-41d012e8838d,Ireland,Flags/IRL.png,Eamonn BOLAND,a8a99c77-b74a-592e-852b-aebea61cc30c,Ireland,Flags/IRL.png,...,30:00.0,4:27:51.3,4:27:51.3,+54:06.0,+42:42.5,RC2,249,459,490,8195
98,01d95c41-ae6e-5b03-9e87-60c0c2be0382,99,#28,15aa0c7e-8fbd-5923-9b70-41d012e8838d,Ireland,Flags/IRL.png,Eamonn BOLAND,a8a99c77-b74a-592e-852b-aebea61cc30c,Ireland,Flags/IRL.png,...,30:00.0,4:27:51.3,4:27:51.3,+54:06.0,,RC2,250,459,490,8195


In [555]:
getOverall(eventId, rallyId, stageId).columns

Index(['id', 'pos', 'carNo', 'driverId', 'driverCountry', 'driverCountryImage',
       'driver', 'coDriverId', 'coDriverCountry', 'coDriverCountryImage',
       'coDriver', 'teamId', 'team/car', 'teamName', 'teamLogo', 'eligibility',
       'stageTime', 'penaltyTime', 'time', 'totalTime', 'diffFirst',
       'diffPrev', 'groupClass', 'championshipId', 'eventId', 'rallyId',
       'stageId'],
      dtype='object')

In [556]:
def getStageTimes(eventId, rallyId, stageId, championship="wrc"):
    stub = f"result/stageTimes?eventId={eventId}&rallyId={rallyId}&stageId={stageId}&championship={championship}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_stageTimes = tablify(json_data)
    stage_id_annotations(df_stageTimes, eventId, rallyId, stageId)
    return df_stageTimes


getStageTimes(eventId, rallyId, stageId).head(3)

,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,teamLogo,eligibility,status,stageId,diffFirst,diffPrev,stageTime,championshipId,eventId,rallyId
0,32fe15b2-3330-5ce8-89fe-4c2838fc0e4d,1,#21,ffedfbf6-eea2-5322-abaf-61c1950f955b,Bulgaria,Flags/BGR.png,Nikolay GRYAZIN,7d63ec40-0bb6-5da3-954e-846a2d40a9a1,ANA,Flags/ANA.png,...,teamLogo/citroen.png,WRC2 (DC/CC),Completed,8195,,,9:12.9,249,459,490
1,08000243-2cce-5380-aa40-77e11c0ee5f8,2,#21,ffedfbf6-eea2-5322-abaf-61c1950f955b,Bulgaria,Flags/BGR.png,Nikolay GRYAZIN,7d63ec40-0bb6-5da3-954e-846a2d40a9a1,ANA,Flags/ANA.png,...,teamLogo/citroen.png,WRC2 (DC/CC),Completed,8195,,,9:12.9,250,459,490
2,5bb89c25-e9fd-5c52-9328-780d9f822fa0,3,#21,ffedfbf6-eea2-5322-abaf-61c1950f955b,Bulgaria,Flags/BGR.png,Nikolay GRYAZIN,7d63ec40-0bb6-5da3-954e-846a2d40a9a1,ANA,Flags/ANA.png,...,teamLogo/citroen.png,WRC2 (DC/CC),Completed,8195,,,9:12.9,251,459,490


In [557]:
#getStageTimes(365, 369, 4161).head(3)

In [558]:
def getSplitTimes(eventId, rallyId, stageId, championship="wrc"):
    stub = f"result/splitTime?eventId={eventId}&rallyId={rallyId}&stageId={stageId}&championship={championship}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_splitTimes = tablify(json_data)
    stage_id_annotations(df_splitTimes, eventId, rallyId, stageId)
    return df_splitTimes


getSplitTimes(eventId, rallyId, stageId).head(3)

,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,round19,round20,round21,round22,round23,round24,round25,eventId,rallyId,stageId
0,980c04b5-cd72-5aaf-849e-51523b96c7ba,1,#21,ffedfbf6-eea2-5322-abaf-61c1950f955b,Bulgaria,Flags/BGR.png,Nikolay GRYAZIN,7d63ec40-0bb6-5da3-954e-846a2d40a9a1,ANA,Flags/ANA.png,...,6:53.8,6:53.8,6:53.8,6:53.8,6:53.8,6:53.8,None,459,490,8195
1,2114e3d0-94d2-5521-81b1-24c79d749104,2,#20,caeac19e-6be8-509e-882a-28ff3a15af6a,Finland,Flags/FIN.png,Sami PAJARI,9a0f092c-8d52-5af3-aa42-3cef8485702b,Finland,Flags/FIN.png,...,+10.7,+10.7,+10.7,+10.7,+10.7,+10.7,None,459,490,8195
2,e7e022cf-8094-5671-b4e4-4fa4ec56e61f,3,#31,e00988f4-2f7f-55f1-8ec6-a0e790578911,Japan,Flags/JPN.png,Hiroki ARAI,1f461eda-7eb3-589b-8694-ecd8f50f538c,Japan,Flags/JPN.png,...,None,None,None,None,None,None,None,459,490,8195


In [559]:
getSplitTimes(eventId, rallyId, stageId).columns

Index(['id', 'pos', 'carNo', 'driverId', 'driverCountry', 'driverCountryImage',
       'driver', 'coDriverId', 'coDriverCountry', 'coDriverCountryImage',
       'coDriver', 'teamId', 'team/car', 'teamName', 'teamLogo', 'eligibility',
       'splitPointId', 'splitPointTimeId', 'groupClass', 'start', 'stageTime',
       'diffFirst', 'round1', 'round2', 'round3', 'round4', 'round5', 'round6',
       'round7', 'round8', 'round9', 'round10', 'round11', 'round12',
       'round13', 'round14', 'round15', 'round16', 'round17', 'round18',
       'round19', 'round20', 'round21', 'round22', 'round23', 'round24',
       'round25', 'eventId', 'rallyId', 'stageId'],
      dtype='object')

In [560]:
championshipId

247

In [561]:
def getStageWinners(eventId, championship="wrc"):
    championshipId = championshipIds[championship]
    stub = f"result/stageWinners?eventId={eventId}&championshipId={championshipId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_stageWinners = tablify(json_data)
    return df_stageWinners


getStageWinners(eventId, ).head(3)

,id,carNo,stageNo,stageName,stageType,stageId,eventId,entryId,driverId,driverCountry,...,coDriverId,coDriverCountry,coDriverCountryImage,coDriver,teamId,team/car,teamName,teamLogo,eligibility,time
0,459,#16,SS1,TOYOTA STADIUM SSS 1 (2.15 km),HeadToHeadSuperSpecialStage,8175,bf0d80b1-6262-538f-8dcb-9b2e3d6a600c,430bdec4-483d-598b-a10a-331d852c61ab,bf956e8a-5cad-5327-add0-26e0481ea508,France,...,8231e16a-f455-56fa-9b3c-dc3befa854cf,France,Flags/FRA.png,Alexandre CORIA,887309d0-48be-5f83-ad3e-abf2a79a64a3,Puma Rally1 HYBRID,Ford,teamLogo/ford.png,M,1:44.4
1,459,#11,SS2,Isegami's Tunnel 1 (23.67 km),SpecialStage,8176,bf0d80b1-6262-538f-8dcb-9b2e3d6a600c,f0cd80af-b964-57a6-94cd-145e2c77d5e3,c99a2a26-bd03-5153-aaa7-684d3acb5491,Belgium,...,b1b98699-0332-528a-8a80-11eb538f1ded,Belgium,Flags/BEL.png,Martijn WYDAEGHE,b6692ea5-df92-5cad-a91c-20319a6fffd7,i20 N Rally1 HYBRID,Hyundai,teamLogo/hyundai.png,M,18:27.4
2,459,#17,SS3,Inabu/Shitara 1 (19.38 km),SpecialStage,8177,bf0d80b1-6262-538f-8dcb-9b2e3d6a600c,16c4777b-6db3-54ca-ad2a-d91d49c769b3,1cf9ade6-25b5-5586-8393-eacfeb943eae,France,...,867cd58f-3fe3-5290-9cc5-b21cb41c523c,France,Flags/FRA.png,Vincent LANDAIS,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,12:14.8


In [562]:
def getItinerary(eventId):
    stub = f"result/itinerary?eventId={eventId}&extended=true"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_itinerary = tablify(json_data, "values")
    return df_itinerary


getItinerary(eventId).tail(3)

,stage,eventId,stageId,distance,timingPrecision,firstCarDueDateTime,firstCarDueDateTimeMs,controlPenalties,status,type,location,targetDuration,targetDurationMs,id,order,date
21,SS21,459,d3063b6a-09e5-5a6f-b574-5bb4bdf557c8,13.98 km,Minute,14:15,2024-11-24T14:15:00+09:00,None,Completed,StageStart,Lake Mikawako 2 Wolf Power Stage,00:07:00,420000,1483.0,4.0,Sunday 24th November
22,SF21,459,d3063b6a-09e5-5a6f-b574-5bb4bdf557c8,,Thousandth,,,None,Completed,FlyingFinish,Lake Mikawako 2 Wolf Power Stage,null,null,1483.0,4.0,Sunday 24th November
23,TC21A,459,null,24.25 km,Minute,15:30,2024-11-24T15:30:00+09:00,Late,Completed,TimeControl,FINISH - Technical Zone & Podium Holding Area IN,01:15:00,4500000,1483.0,4.0,Sunday 24th November


In [563]:
def getStartlist(eventId):
    stub = f"result/startLists?eventId={eventId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_startlist = tablify(json_data, "startListItems")
    return df_startlist


getStartlist(eventId).tail(3)

,order,startDateTimeLocal,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,coDriver,teamId,team/car,teamName,teamLogo,eligibility,groupClass,priority,id,date
40,41,2024-11-24T06:21:00+09:00,#54,e2464489-5926-5ac0-94a3-bdbf823bb827,Japan,Flags/JPN.png,Yasunori HAGIWARA,86332dbd-c66e-5802-afc0-a38dd5c72128,Japan,Flags/JPN.png,Masami NAKATA,1d513c0b-cbe3-5b2e-b440-e5e14d0b22ef,WRX STI 2015,Subaru,teamLogo/subaru.png,,JR1,None,f838dad2-9943-5dec-92d8-04b24016d304,Sunday 2024-11-24
41,42,2024-11-24T06:21:00+09:00,#53,8584c859-11be-5de8-9ba1-f91976ec0b46,Japan,Flags/JPN.png,Satoshi IRIE,7336f886-56da-5a98-b9a8-fc626172e583,Japan,Flags/JPN.png,Satoshi KAGEYAMA,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,Vitz,Toyota,teamLogo/toyota.png,,JR3,None,f838dad2-9943-5dec-92d8-04b24016d304,Sunday 2024-11-24
42,43,2024-11-24T06:21:00+09:00,#56,3ab588df-cc06-5017-8ba6-143de81a5972,Japan,Flags/JPN.png,Koji TAKATA,98398924-b2b8-51b7-82fe-d2501f5c4b0d,Japan,Flags/JPN.png,Hiroki YAMAGUCHI,be461a0c-d1fd-5052-a69c-3fd94f8cf5f6,GR Yaris,Toyota,teamLogo/toyota.png,,JR1,None,f838dad2-9943-5dec-92d8-04b24016d304,Sunday 2024-11-24


In [564]:
def getPenalties(eventId):
    stub = f"result/penalty?eventId={eventId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_penalties = tablify(json_data)
    return df_penalties


getPenalties(eventId).head(3)

,id,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,coDriver,...,eligibility,groupClass,rallyId,entryId,controlId,penaltyTime,penaltyDuration,penaltyDurationMs,reason,control
0,7066,#49,6a8d7e7e-de6b-53a3-8173-7b8be7a43f3e,Japan,Flags/JPN.png,Mako HIRAKAWA,e8d4da75-7517-5be5-a3b8-5abfea5141fc,Japan,Flags/JPN.png,Kazuki ISHIDA,...,,JR3,490,54580,34603,10.0,PT10S,10000,1 MIN LATE,TC2
1,7067,#54,e2464489-5926-5ac0-94a3-bdbf823bb827,Japan,Flags/JPN.png,Yasunori HAGIWARA,86332dbd-c66e-5802-afc0-a38dd5c72128,Japan,Flags/JPN.png,Masami NAKATA,...,,JR1,490,54585,34603,40.0,PT40S,40000,4 MINS LATE,TC2
2,7068,#54,e2464489-5926-5ac0-94a3-bdbf823bb827,Japan,Flags/JPN.png,Yasunori HAGIWARA,86332dbd-c66e-5802-afc0-a38dd5c72128,Japan,Flags/JPN.png,Masami NAKATA,...,,JR1,490,54585,34609,50.0,PT50S,50000,5 MINS LATE,TC4


In [565]:
def getRetirements(eventId):
    stub = f"result/retirements?eventId={eventId}"
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_retirements = tablify(json_data)
    return df_retirements


getRetirements(eventId).head(3)

,id,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,coDriver,...,team/car,teamName,teamLogo,eligibility,groupClass,rallyId,entryId,controlId,reason,control
0,8817,#44,0a4d6b0a-f926-56f3-aca2-049f93add127,Japan,Flags/JPN.png,Tomoyuki SHINKAI,8e4b14a8-433b-5001-b586-0f5cf15edf1f,Japan,Flags/JPN.png,Yuichi ANDO,...,GR Yaris,Toyota,teamLogo/toyota.png,,JR1,490,54575,34603,MECHANICAL,TC2
1,8818,#51,53f655b1-91e6-5163-adad-f8e7e4c6a77b,Japan,Flags/JPN.png,Mitsuhiro KUNISAWA,b57311ff-0e08-5f28-8806-efb80bfd6356,Japan,Flags/JPN.png,Masahiko KIHARA,...,Clio RS Line,Renault,teamLogo/renault.png,,RC5,490,54582,34605,MECHANICAL,SF2
2,8819,#38,b487d175-4d7a-563c-bf10-8ec3dc5f0bc3,Japan,Flags/JPN.png,Satoshi IMAI,ce5d6a41-d103-51fe-8b44-2cb3776f8831,New Zealand,Flags/NZL.png,Jason FARMER,...,C3,Citroen,teamLogo/citroen.png,WRC2 (DCM/CC),RC2,490,54570,34608,ACCIDENT,SF3


In [566]:
def getChampionship(
    seasonId, eventId, championship_type="driver", championship="all", retUrl=False
):
    """ championship_tpe: driver | codriver | manufacturer"""
    championshipId = championshipIds[championship]
    stub = f"result/championshipresult?seasonId={seasonId}&championshipId={championshipId}&type={championship_type}&championship={championship}"
    if retUrl:
        return stub
    json_data = _WRC_json(stub)
    if not json_data:
        return pd.DataFrame()
    df_splitTimes = tablify(json_data)
    return df_splitTimes


championship_type = "driver"  # where is list?

getChampionship(seasonId, championship_type).head(3)

,championshipEntryId,overallPosition,overallPoints,driver,driverCountryImage,driverId,tyreManufacturer,teamName,teamLogo,_contry_iso_MCO,...,pointsBreakDown_contry_iso_EUR,position_contry_iso_EUR,status_contry_iso_EUR,totalPoints_contry_iso_EUR,_contry_iso_JPN,dropped_contry_iso_JPN,pointsBreakDown_contry_iso_JPN,position_contry_iso_JPN,status_contry_iso_JPN,totalPoints_contry_iso_JPN
0,5144,1,242,Thierry NEUVILLE,Flags/BEL.png,c99a2a26-bd03-5153-aaa7-684d3acb5491,None,Hyundai,teamLogo/hyundai.png,,...,13 + 3 + 2,(3),Finished,18,,false,6 + 7 + 4,(6),Finished,17
1,5146,2,210,Elfyn EVANS,Flags/GBR.png,ae7329c9-79b3-5d96-886c-22cca6217764,None,Toyota,teamLogo/toyota.png,,...,15 + 6 + 3,(2),Finished,24,,false,18 + 4 + 3,(1),Finished,25
2,5147,3,200,Ott TÄNAK,Flags/EST.png,6632e7ca-34bf-55b8-9cad-d060000fa794,None,Hyundai,teamLogo/hyundai.png,,...,18 + 4,(1),Finished,22,,false,R,(R),RetiredAccident,0


In [567]:
getChampionship(seasonId, eventId, championship_type)
#championshipresult?seasonId=28&championshipId=245&type=driver&championship=wrc

,championshipEntryId,overallPosition,overallPoints,driver,driverCountryImage,driverId,tyreManufacturer,teamName,teamLogo,_contry_iso_MCO,...,pointsBreakDown_contry_iso_EUR,position_contry_iso_EUR,status_contry_iso_EUR,totalPoints_contry_iso_EUR,_contry_iso_JPN,dropped_contry_iso_JPN,pointsBreakDown_contry_iso_JPN,position_contry_iso_JPN,status_contry_iso_JPN,totalPoints_contry_iso_JPN
0,5144,1,242,Thierry NEUVILLE,Flags/BEL.png,c99a2a26-bd03-5153-aaa7-684d3acb5491,None,Hyundai,teamLogo/hyundai.png,,...,13 + 3 + 2,(3),Finished,18,,false,6 + 7 + 4,(6),Finished,17
1,5146,2,210,Elfyn EVANS,Flags/GBR.png,ae7329c9-79b3-5d96-886c-22cca6217764,None,Toyota,teamLogo/toyota.png,,...,15 + 6 + 3,(2),Finished,24,,false,18 + 4 + 3,(1),Finished,25
2,5147,3,200,Ott TÄNAK,Flags/EST.png,6632e7ca-34bf-55b8-9cad-d060000fa794,None,Hyundai,teamLogo/hyundai.png,,...,18 + 4,(1),Finished,22,,false,R,(R),RetiredAccident,0
3,5145,4,191,Sébastien OGIER,Flags/FRA.png,1cf9ade6-25b5-5586-8393-eacfeb943eae,None,Toyota,teamLogo/toyota.png,,...,R,(R),RetiredAccident,0,,false,15 + 5 + 5,(2),Finished,25
4,5148,5,162,Adrien FOURMAUX,Flags/FRA.png,bf956e8a-5cad-5327-add0-26e0481ea508,None,Ford,teamLogo/ford.png,,...,0 + 5 + 1,(32),Finished,6,,false,13 + 3,(3),Finished,16
5,5150,6,116,Takamoto KATSUTA,Flags/JPN.png,298f93b1-b0ef-5af4-9f0c-e468d29abfd2,None,Toyota,teamLogo/toyota.png,,...,10 + 7 + 5,(4),Finished,22,,false,10 + 2 + 2,(4),Finished,14
6,5273,7,114,Kalle ROVANPERÄ,Flags/FIN.png,d8e4bbea-3af2-5486-9ad5-a445aaec573e,None,Toyota,teamLogo/toyota.png,,...,-,(-),DidNotEnter,0,,false,-,(-),DidNotEnter,0
7,5154,8,46,Grégoire MUNSTER,Flags/LUX.png,c718f17c-f6a0-54b5-bad9-a406b8cc95f0,None,Ford,teamLogo/ford.png,,...,8 + 2,(5),Finished,10,,false,8 + 1,(5),Finished,9
8,5870,9,44,Daniel SORDO,Flags/ESP.png,d06b6c54-863a-533d-9241-9dd04f7acb85,None,Hyundai,teamLogo/hyundai.png,,...,-,(-),DidNotEnter,0,,false,-,(-),DidNotEnter,0
9,5276,10,44,Sami PAJARI,Flags/FIN.png,caeac19e-6be8-509e-882a-28ff3a15af6a,None,Toyota,teamLogo/toyota.png,,...,R,(R),RetiredAccident,0,,false,3 + 0,(8),Finished,3


In [582]:
def time_to_seconds(time_str):
    """
    Convert a time string in the format 'MM:SS.T' to total seconds including tenths (1 dp).

    Parameters:
        time_str (str): Time string in the format 'MM:SS.T'.

    Returns:
        float: Total time in seconds rounded to 1 decimal place.
    """
    if not time_str or not isinstance(time_str, str):  # Handle empty or invalid input
        return None
    
    try:
        # Split the time string into parts
        parts = time_str.split(':')
        
        # Depending on the number of parts, interpret hours, minutes, and seconds
        if len(parts) == 3:  # Hours, minutes, seconds.tenths
            hours, minutes, seconds = parts
            total_seconds = int(hours) * 3600 + int(minutes) * 60 + float(seconds)
        elif len(parts) == 2:  # Minutes, seconds.tenths
            minutes, seconds = parts
            total_seconds = int(minutes) * 60 + float(seconds)
        else:
            seconds = parts[0]
            total_seconds = float(seconds)
        
        # Round to 1 decimal place
        return round(total_seconds, 1)
    except:
        print(f"Invalid time format: {time_str}")



# Function to apply time delta
def apply_time_delta(base_time_str, delta_str):
    # Convert base time and delta time to seconds
    base_seconds = time_to_seconds(base_time_str)
    delta_seconds = time_to_seconds(delta_str)

    if base_seconds is None or delta_seconds is None:
        return None

    # If delta is positive, add, if negative, subtract
    if delta_str.startswith('-'):
        return round(base_seconds - delta_seconds, 1)
    else:
        return round(base_seconds + delta_seconds, 1)

In [583]:
import sqlite_utils
#!rm wrc_2024_archive_results.db
_db_filepath = f"wrc_{CURRENT_YEAR}_archive_results.db"
try:
    pathlib.Path.unlink(_db_filepath)
except:
    pass
# db = sqlite_utils.Database("wrc_2023_results.db")

#!rm wrc_2024_12_results.db
db = sqlite_utils.Database(_db_filepath)

In [584]:
# db["delme"].insert_all(df_fullcal.to_dict("records"))
# db["delme"].drop()

In [585]:
def db_add(table, df):
    if not df.empty:
        db[table].insert_all(df.to_dict("records"), alter=True)

db_add("fullcal", getFullCalendar())
db_add("resultscal", getResultsCalendar())

In [586]:
# We can add alter=True to dynamically modify tables

def drop_ephemera_cols(df):
    try:
        cols = ["driverCountry", "driverCountryImage", "coDriverCountry","coDriverCountryImage", "teamLogo" ]
        dropcols = [col for col in cols if col in df.columns]
        df.drop(columns=dropcols, inplace=True)
    except:
        print("Can't drop cols...")
def splits_long(df):    
    # Identify the 'roundN' columns dynamically
    round_columns = [col for col in df.columns if col.startswith('round')]
    
    # Use pd.melt to transform the DataFrame into a long format
    long_df = pd.melt(
        df,
        #id_vars=[col for col in df.columns if col not in round_columns],  # Keep all columns except 'roundN' as identifiers
        id_vars=["pos", "carNo", "driverId", "driver", 	"splitPointId", "splitPointTimeId", "groupClass", "start","stageTime","diffFirst","eventId","rallyId","stageId"],
        value_vars=round_columns,  # Columns to unpivot
        var_name='round',          # Name of the new column for round labels
        value_name='time'          # Name of the new column for the round values
    )
    
    # Extract the N value from the 'round' column
    long_df['round_number'] = long_df['round'].str.extract('(\d+)$').astype(int)
    
    # Drop the original 'round' column if needed (optional)
    long_df = long_df.drop(columns=['round'])

    long_df['total_seconds'] = long_df['stageTime'].apply(time_to_seconds)

    # Process the time deltas based on the first row
    base_time = long_df.loc[0, 'time']  # The base time is in the first row
    # Apply the time deltas starting from the second row
    long_df['time_s'] = long_df['time'].apply(lambda x: apply_time_delta(base_time, x))

    return long_df

def stageresults(row, eventId, rallyId):
    stageId = row["stageId"]
    if stageId=="FINAL":
        return
    try:
        _df_stageTimes = getStageTimes(eventId, rallyId, stageId)
        drop_ephemera_cols(_df_stageTimes)
        if 'event' in _df_stageTimes.columns:
            db_add("shakedown_stagetimes", _df_stageTimes)   
        else:
            db_add("stagetimes", _df_stageTimes)
    except:
        print(f"Couldn't get stagetimes for {eventId} {rallyId} {stageId}")
    try:
        _df_splitTimes = getSplitTimes(eventId, rallyId, stageId)
        drop_ephemera_cols(_df_splitTimes)
        if 'event' in _df_splitTimes.columns:
            db_add("shakedown_split", _df_splitTimes)   
        else:
            db_add("splittimes", _df_splitTimes)
            try:
                db_add("splittimes_long",splits_long(_df_splitTimes))
            except:
                print(f"Couldn't make {eventId} {rallyId} {stageId} splits long")
    except:
          print(f"Couldn't get splittimes for {eventId} {rallyId} {stageId}")
    try:
        _df_Overall = getOverall(eventId, rallyId, stageId)
        if 'event' in _df_Overall.columns:
            db_add("shakedown_overall", _df_Overall)   
        else:
            db_add("overall", _df_Overall)
    except:
        print(f"Couldn't get overall for {eventId} {rallyId} {stageId}")


def rallyresults(row):
    eventId, rallyId = row[["eventId", "rallyId"]]
    db_add("retirements",getRetirements(eventId))
    db_add("penalties",getPenalties(eventId))
    db_add("startlist",getStartlist(eventId))
    db_add("itinerary",getItinerary(eventId))
    db_add("stagewinners",getStageWinners(eventId))

    df_stageDetails = getStageDetails(eventId, rallyId)
    db_add("stagedetails", df_stageDetails)
    df_stageDetails.apply(stageresults, axis=1, args=(eventId, rallyId))
    #stageresults(df_stageDetails)


def rallyresults2(row):
    eventId, rallyId = row[["eventId", "rallyId"]]
    print(eventId, rallyId, row["startDateLocal"])

_ = df_fullcal.apply(rallyresults, axis=1)

Couldn't make 459 490 8175 splits long


In [581]:
_ = df_fullcal.apply(rallyresults2, axis=1)

446 477 2024-01-24T16:30:00+01:00
447 478 2024-02-15T09:00:00+01:00
448 479 2024-03-27T10:00:00+03:00
449 480 2024-04-18T09:00:00+02:00
450 481 2024-05-09T08:00:00+01:00
452 483 2024-05-30T18:45:00+02:00
453 484 2024-06-27T10:00:00+02:00
454 485 2024-07-18T07:31:00+03:00
455 486 2024-07-31T19:00:00+03:00
456 487 2024-09-05T09:01:00+03:00
457 488 2024-09-26T08:31:00-03:00
458 489 2024-10-17T09:30:00+02:00
459 490 2024-11-21T09:00:00+09:00


In [95]:
df_fullcal.columns

Index(['id', 'guid', 'title', 'location', 'startDate', 'endDate', 'eventId',
       'rallyId', 'description', 'round', 'cvpSeriesLink', 'sponsor', 'images',
       'season', 'competition', 'country', 'asset', '__typename', 'type',
       'uid', 'seriesUid', 'releaseYear', 'availableOn', 'availableTill',
       'startDateLocal', 'endDateLocal', 'finishDate', 'championship',
       'championshipLogo'],
      dtype='object')

In [96]:
getStageTimes(eventId, rallyId, stageId)

,id,pos,carNo,driverId,driverCountry,driverCountryImage,driver,coDriverId,coDriverCountry,coDriverCountryImage,...,team/car,teamName,teamLogo,eligibility,status,stageId,diffFirst,diffPrev,stageTime,championshipId
0,f0e87af5-0cfa-5c94-a792-6106abeacde9,1,#11,c99a2a26-bd03-5153-aaa7-684d3acb5491,Belgium,Flags/BEL.png,Thierry NEUVILLE,b1b98699-0332-528a-8a80-11eb538f1ded,Belgium,Flags/BEL.png,...,i20 N Rally1 HYBRID,Hyundai,teamLogo/hyundai.png,M,Completed,6140,,,4:48.8,None
1,e3b0a0d6-ffcd-597d-8810-c9cc5e31cc13,2,#8,6632e7ca-34bf-55b8-9cad-d060000fa794,Estonia,Flags/EST.png,Ott TÄNAK,00a8a5c3-f7ba-5086-86df-1a59b7da7e26,Estonia,Flags/EST.png,...,Puma Rally1 HYBRID,Ford,teamLogo/ford.png,M,Completed,6140,+2.4,+2.4,4:51.2,None
2,f2f3da23-d724-52a9-bd01-362e0e389e81,3,#4,916f5b10-fee9-5b4c-b17b-6fbbc343cc3c,Finland,Flags/FIN.png,Esapekka LAPPI,2ef28d31-4a74-5794-b720-12c327c941b6,Finland,Flags/FIN.png,...,i20 N Rally1 HYBRID,Hyundai,teamLogo/hyundai.png,M,Completed,6140,+6.8,+4.4,4:55.6,None
3,4965f409-4d74-52b2-9a6b-c97db36bd1e7,4,#18,298f93b1-b0ef-5af4-9f0c-e468d29abfd2,Japan,Flags/JPN.png,Takamoto KATSUTA,6347a621-eade-5fff-a7b4-7c4b73651e8a,Ireland,Flags/IRL.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,,Completed,6140,+7.1,+0.2,4:55.9,None
4,c58f0ad5-6dc9-5f3a-ab24-6d3306c89e06,5,#17,1cf9ade6-25b5-5586-8393-eacfeb943eae,France,Flags/FRA.png,Sébastien OGIER,867cd58f-3fe3-5290-9cc5-b21cb41c523c,France,Flags/FRA.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,Completed,6140,+7.3,+0.2,4:56.1,None
5,dee62ffc-9830-5564-97fd-b85d47ce61d2,6,#69,d8e4bbea-3af2-5486-9ad5-a445aaec573e,Finland,Flags/FIN.png,Kalle ROVANPERÄ,e4dd8a3f-00e9-59f7-9871-9337af6085d7,Finland,Flags/FIN.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,Completed,6140,+9.9,+2.6,4:58.7,None
6,79c1931c-0cab-5b80-bdaa-d8674c15997e,7,#33,ae7329c9-79b3-5d96-886c-22cca6217764,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,Elfyn EVANS,53e56691-fe7c-5271-9dc5-8960df28b221,United Kingdom of Great Britain and Northern I...,Flags/GBR.png,...,GR Yaris Rally1 HYBRID,Toyota,teamLogo/toyota.png,M,Completed,6140,+13.4,+3.5,5:02.2,None
7,795c4177-f68a-57a3-ad17-d4d2851e1dbe,8,#21,199a29d0-97f5-530f-9a5d-7ab45b1da0cc,ANA,Flags/ANA.png,Nikolay GRYAZIN,7d63ec40-0bb6-5da3-954e-846a2d40a9a1,ANA,Flags/ANA.png,...,Fabia RS,Skoda,teamLogo/skoda.png,WRC2 (T/DC/CC),Completed,6140,+21.4,+8.1,5:10.2,None
8,c70c9ad6-e0bc-5ed7-a339-c2ace0b02885,9,#22,cede2cae-3446-54d8-b9d7-7e65651ebfd1,Poland,Flags/POL.png,Kajetan KAJETANOWICZ,a488aa5c-affc-5f48-a95e-0f400495fd71,Poland,Flags/POL.png,...,Fabia RS,Skoda,teamLogo/skoda.png,WRC2 (DC/CC),Completed,6140,+29.7,+8.3,5:18.6,None
9,a0977e64-2959-5722-8be1-ab4ea8573a75,10,#20,093f0b06-7a05-5353-92ac-f5a68ae51f41,Norway,Flags/NOR.png,Andreas MIKKELSEN,5d0fb669-fb61-5d51-805b-d09c1ac7e36a,Norway,Flags/NOR.png,...,Fabia RS,Skoda,teamLogo/skoda.png,WRC2 (T/D),Completed,6140,+30.2,+0.5,5:19.0,None


In [97]:
for row in db.query("select DISTINCT(eventId) from stagedetails "):
    print(row)

{'eventId': '353'}
{'eventId': '354'}
{'eventId': '355'}
{'eventId': '356'}
{'eventId': '357'}
{'eventId': '358'}
{'eventId': '359'}
{'eventId': '360'}
{'eventId': '361'}
{'eventId': '362'}
{'eventId': '363'}
{'eventId': '364'}
{'eventId': '365'}


In [42]:
!ls -al wrc_2024_12_results.db

-rw-r--r--  1 tonyhirst  staff  3108864 27 Nov 00:07 wrc_2024_12_results.db


In [114]:
db.table_names()

['fullcal',
 'resultscal',
 'retirements',
 'penalties',
 'startlist',
 'itinerary',
 'stagedetails',
 'stagetimes',
 'splittimes',
 'overall']